# Elementos Finitos en Torch

In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
import torch
import torchgeometry

Definimos la geometria de la placa y la malla de elementos finitos:

In [3]:
M = 50  # número de nodos en la dirección x
L = 100  # número de nodos en la dirección y

# Tamaño de los elementos
delta_x = 1.0 / (M-1)  # tamaño del paso en la dirección x
delta_y = 1.0 / (L-1)  # tamaño del paso en la dirección y

# Definir la geometría de la placa rectangular
x = torch.linspace(0, M-1, M) * delta_x
y = torch.linspace(0, L-1, L) * delta_y
# X, Y = torchgeometry.meshgrid(x, y)
X, Y = torch.meshgrid(x, y)

# Crear la malla de elementos finitos uniforme en la placa rectangular
nodes = torch.stack((X.reshape(-1), Y.reshape(-1)), dim=1)
elems = torch.zeros((2, (M-1)*(L-1), 3), dtype=torch.long)
idx = 0
for i in range(L-1):
    for j in range(M-1):
        p1 = i*M + j
        p2 = i*M + j + 1
        p3 = (i+1)*M + j
        p4 = (i+1)*M + j + 1
        elems[:, idx, :] = torch.tensor([[p1, p2, p3], [p4, p3, p2]])
        idx += 1

c:\Users\Det-Pc\anaconda3\envs\pytorch-C-DL\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Definir la función de forma y la matriz de rigidez:

In [4]:
# # Definir la función de forma (usando elementos lineales)
# def shape_functions(p):
#     """Calcular las funciones de forma para un punto p."""
#     N = torch.zeros(3)
#     N[0] = 1.0 - p[0] - p[1]
#     N[1] = p[0]
#     N[2] = p[1]
#     return N

# # # Definir la matriz de rigidez y el vector de carga para cada elemento
# # K_elem = torch.zeros((3, 3))
# # K_elem[0, :] = torch.tensor([1, -1, 0])
# # K_elem[1, :] = torch.tensor([-1, 1, 0])
# # K_elem[2, :] = torch.tensor([0, 0, 0.5])
# # K_elem *= 1.0  # conductividad térmica
# # K_elem *= delta_x * delta_y / 2.0  # tamaño del elemento
# # F_elem = torch.zeros(3)
# # F_elem[2] = 1.0  # fuente de calor uniforme
# # F_elem *= delta_x * delta_y / 3.0  # tamaño del elemento
# # Definir la matriz de rigidez y el vector de carga para cada elemento
# K_elem = torch.zeros((3, 3))
# K_elem[0, :] = torch.tensor([1, -1, 0])
# K_elem[1, :] = torch.tensor([-1, 1, 0])
# K_elem[2, :] = torch.tensor([0, 0, 0.5])
# K_elem *= 1.0  # conductividad térmica
# K_elem *= delta_x * delta_y / 2.0  # tamaño del elemento
# F_elem = torch.zeros(3)
# F_elem[2] = 1.0  # fuente de calor uniforme
# F_elem *= delta_x * delta_y / 3.0  # tamaño del elemento

# # # Definir la matriz de rigidez y el vector de carga para toda la placa
# # K = torch.zeros((M*L, M*L))
# # F = torch.zeros(M*L)
# # for i in range(elems.shape[1]):
# #     p1 = nodes[elems[0, i, 0]]
# #     p2 = nodes[elems[0, i, 1]]
# #     p3 = nodes[elems[0, i, 2]]
# #     J = torch.stack((p2-p1, p3-p1), dim=1)
# #     J_inv = torch.inverse(J)
# #     detJ = torch.det(J)
# #     B = J_inv @ torch.tensor([[-1, 1, 0], [-1, 0, 1]])
# #     K_e = B.T @ K_elem @ B * detJ
# #     F_e = shape_functions(torch.tensor([1/3, 1/3])) @ F_elem * detJ
# #     for a in range(3):
# #         for b in range(3):
# #             i_a = elems[0, i, a]
# #             i_b = elems[0, i, b]
# #             K[i_a, i_b] += K_e[a, b]
# #         F[i_a] += F_e[a]
# # Definir la matriz de rigidez y el vector de carga para toda la placa
# K = torch.zeros((M*L, M*L))
# F = torch.zeros(M*L)
# for i in range(elems.shape[1]):
#     p1 = nodes[elems[0, i, 0]]
#     p2 = nodes[elems[0, i, 1]]
#     p3 = nodes[elems[0, i, 2]]
#     J = torch.stack((p2-p1, p3-p1), dim=1)
#     detJ = torch.det(J)
#     if detJ == 0:
#         continue
#     J_inv = torch.inverse(J)
#     # # # B = J_inv @ torch.tensor([[-1, 1, 0], [-1, 0, 1]])
    
#     B = J_inv @ torch.tensor([[-1, 1, 0], [-1, 0, 1]], dtype=torch.float32)
#     K_elem = B @ K_elem @ B.T * detJ
    
#     # # B = J_inv @ torch.tensor([[-1, 1], [-1, 0], [0, 1]], dtype=torch.float32)
#     # # K_elem = B.T @ K_elem.T @ B * detJ
#     # B = J_inv @ torch.tensor([[-1, 1], [-1, 0], [0, 1]], dtype=torch.float32)
#     # K_elem = B.T @ K_elem.T @ B * detJ
    
#     # B = J_inv @ torch.tensor([[-1, -1], [1, 0], [0, 1]], dtype=torch.float32)
#     # K_elem = B.T @ K_elem @ B * detJ

#     # F_elem_local = shape_functions(torch.mean(torch.stack((p1, p2, p3)), dim=0)) * F_elem
#     F_elem_local = shape_functions(torch.mean(torch.stack((p1, p2, p3)), dim=0)) * F_elem
#     F_elem_global = torch.zeros(M*L)
#     node_indices = elems[0, i]
#     for j in range(3):
#         F_elem_global[2*node_indices[j]:2*node_indices[j]+2] += F_elem_local[j*2:j*2+2]
#     K[node_indices.repeat(2), node_indices.repeat(2, 1)] += K_elem
#     F[node_indices.repeat(2)] += F_elem_global


In [8]:
# Definir la función de forma (usando elementos lineales)
def shape_functions(p):
    """Calcular las funciones de forma para un punto p."""
    N = torch.zeros(3)
    N[0] = 1.0 - p[0] - p[1]
    N[1] = p[0]
    N[2] = p[1]
    return N

# Definir la matriz de rigidez y el vector de carga para cada elemento
K_elem = torch.zeros((3, 3))
K_elem[0, :] = torch.tensor([1, -1, 0], dtype=torch.float32)
K_elem[1, :] = torch.tensor([-1, 1, 0], dtype=torch.float32)
K_elem[2, :] = torch.tensor([0, 0, 0.5], dtype=torch.float32)
K_elem *= 1.0  # conductividad térmica
K_elem *= delta_x * delta_y / 2.0  # tamaño del elemento
F_elem = torch.zeros(3, dtype=torch.float32)
F_elem[2] = 1.0  # fuente de calor uniforme
F_elem *= delta_x * delta_y / 3.0  # tamaño del elemento

# Definir la matriz de rigidez y el vector de carga para toda la placa
K = torch.zeros((M*L, M*L), dtype=torch.float32)
F = torch.zeros(M*L, dtype=torch.float32)
for i in range(elems.shape[1]):
    p1 = nodes[elems[0, i, 0]]
    p2 = nodes[elems[0, i, 1]]
    p3 = nodes[elems[0, i, 2]]
    J = torch.stack((p2-p1, p3-p1), dim=1)
    detJ = torch.det(J)
    if detJ == 0:
        continue
    J_inv = torch.inverse(J)
    
    B = J_inv @ torch.tensor([[-1, 1, 0], [-1, 0, 1]], dtype=torch.float32)
    K_elem_local = B @ K_elem @ B.T * detJ
    F_elem_local = shape_functions(torch.mean(torch.stack((p1, p2, p3)), dim=0)) * F_elem
    F_elem_global = torch.zeros(M*L, dtype=torch.float32)
    node_indices = elems[0, i]
    print(node_indices.shape)
    for j in range(3):
        F_elem_global[2*node_indices[j]:2*node_indices[j]+2] += F_elem_local[j]
        for k in range(3):
            K[2*node_indices[j]:2*node_indices[j]+2, 2*node_indices[k]:2*node_indices[k]+2] += K_elem_local[j, k]
        F[2*node_indices[j]:2*node_indices[j]+2] += F_elem_local[j]


torch.Size([3])


IndexError: index 2 is out of bounds for dimension 1 with size 2